In [30]:
import pandas as pd
import numpy as np 
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import random

In [29]:
N = 3 # tunable?

In [2]:
drug_response_data = pd.read_csv("/data/yingfei/cancer_data/cell_drug_auc_final_1111.csv")
print(drug_response_data.shape)
drug_response_data.head()

(692, 386)


,ARXSPAN_ID,JW-7-24-1,KIN001-260,NSC-87877,GNE-317,NAVITOCLAX,PLX-4720,ERK5-IN-1,VX-11E,TGX-221,...,BIX02189,ISPINESIB MESYLATE,KIN001-135,KIN001-206,KIN001-236,KIN001-266,LUMINESPIB,NUTLIN-3A,SGC0946,SL 0101-1
0,ACH-000001,0.778432,0.951321,0.840287,-99999.000000,-99999.000000,0.936410,0.891908,0.402122,0.659600,...,0.982272,0.733492,0.072277,0.983536,-99999.000000,0.651746,0.982084,0.893777,0.983861,0.933035
1,ACH-000002,0.788327,0.874760,0.760137,0.905754,0.930764,0.991068,0.278288,0.972905,0.558774,...,0.899628,0.604228,0.046061,0.918265,0.865628,-99999.000000,0.985921,0.839175,0.988639,0.533204
2,ACH-000004,0.737630,0.938733,0.835441,0.808964,0.966133,0.983552,-99999.000000,0.993912,0.596027,...,0.965245,0.782295,0.366912,0.844979,0.761323,-99999.000000,0.980945,0.946694,0.992605,0.761956
3,ACH-000006,0.176396,0.571569,0.581729,0.585938,0.958510,0.935574,0.600428,-99999.000000,0.596056,...,0.834158,0.605360,0.124930,0.813384,0.782187,-99999.000000,0.971058,0.871277,-99999.000000,0.495020
4,ACH-000007,0.569751,0.755449,0.954924,0.939587,0.965638,0.932666,-99999.000000,0.956481,0.838692,...,0.927519,0.812955,0.719780,0.815099,0.795736,0.860768,0.983317,0.888067,0.987564,0.787948


In [3]:
cell_line_mutation = pd.read_csv("/data/yingfei/cancer_data/mutations_raw_324_final.csv")
print(cell_line_mutation.shape)
cell_line_mutation.head()

(692, 325)


,DepMap_ID,ABL1,ACVR1B,AKT1,AKT2,AKT3,ALK,ALOX12B,FAM123B,APC,...,VEGFA,VHL,WHSC1,WHSC1L1,WT1,XPO1,XRCC2,ZNF217,ZNF703,TERC
0,ACH-000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
1,ACH-000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,ACH-000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,ACH-000006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,ACH-000007,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [4]:
drug_smiles_data = pd.read_csv("/data/yingfei/cancer_data/drug_smiles.csv", header = None, names = ['drug_name', 'smiles'])
print(drug_smiles_data.shape)
drug_smiles_data.head()

(408, 2)


,drug_name,smiles
0,JW-7-24-1,COC1=CC(=CC(=C1)C2=CC3=C4C(=CN=C3C=C2)C=CC(=O)...
1,KIN001-260,C1CC1COC2=CC=CC(=O)C2=C3C=C(C(=C(N3)N)C#N)C4CC...
2,NSC-87877,C1=CC2=C(C(=O)C(=NNC3=CC4=C(C=C3)C=C(C=C4)S(=O...
3,GNE-317,CC1=C(SC2=C1N=C(N=C2N3CCOCC3)C4=CN=C(N=C4)N)C5...
4,NAVITOCLAX,CC1(CCC(=C(C1)CN2CCN(CC2)C3=CC=C(C=C3)C(=O)NS(...


In [5]:
### drug recommendation task [given the prompt with a cell line and N candidate drugs -> predict the most suitable drug and provide reasoning]

### Input Prompt:
# "Think step by step and decide the best drug option for the cell line with given mutations: [Drug Name], [Reasoning]
# Drug 1: The drug is AGI-6780. The drug SMILES structure is … Drug Target pathway: Metabolism. 
# Drug 2: The drug is JW-7-24-1. The drug SMILES structure is … Drug Target pathway: Other, kinases. 
# Drug 3: The drug is KIN001-260. The drug SMILES structure is …Drug Target pathway: DNA replication. 
# The mutations of the cell lines are NOTCH1 NOTCH3 PIK3R1 PPP2R1A TP53 TSC2 WHSCL1. 
# Best drug option: ?
### Output Groundtruth: ?

### Data needed for prompt: cell line mutation data, drug information (drug name, drug SMILES structure, target pathway [KEGG/database])
### Data needed for output: cell line drug response prediction (drug with lower AUDRC value indicates better response)

### Intermediate dataset: Cell line name, mutation status columns, Drug 1 name, Drug 1 SMILES, Drug 1 target pathway, Drug 1 response, Drug 2 name, ...
### Initiliaze the data
interm_data = {
    'cell_id': [],
    'mut_status': []
}

for i in range(N):
    drug_id = f'drug_{i+1}'
    interm_data[f'{drug_id}_name'] = []
    interm_data[f'{drug_id}_SMILES'] = []
    interm_data[f'{drug_id}_target'] = []
    interm_data[f'{drug_id}_target_path'] = []
    interm_data[f'{drug_id}_response'] = []

In [6]:
### Fill in data features
gene_columns = cell_line_mutation.columns[1:]
interm_data['cell_id'] = cell_line_mutation['DepMap_ID']
interm_data['mut_status'] = cell_line_mutation.apply(lambda row: ', '.join(gene for gene in gene_columns if row[gene] == 1), axis=1)
top_n_drugs = drug_response_data.drop('ARXSPAN_ID', axis=1).apply(lambda row: ', '.join(row[row > 0].nsmallest(N).index), axis=1)
top_n_drug_response = drug_response_data.drop('ARXSPAN_ID', axis=1).apply(lambda row: list(row[row > 0].nsmallest(3)), axis=1)
for n in range(N):
    drug_id = f"drug_{n+1}"
    interm_data[f'{drug_id}_name'] = top_n_drugs.apply(lambda x: x.split(', ')[n])
    interm_data[f'{drug_id}_response'] = top_n_drug_response.apply(lambda x: x[n])

In [7]:
def find_drug_page(drug_name):
    url = f'https://www.cancerrxgene.org/search?query={drug_name}'
    response = requests.get(url)
    
    if response.ok:
        try:
            soup = BeautifulSoup(response.text, 'html.parser')
            link_tag = soup.table.find_all('a', href=True)
            link = link_tag[0]['href']
            return link
        except Exception as e:
            # print(f"Failed to retrieve the web page for drug {drug_name}, {e}")
            return ""

In [8]:
def drug_info_ws(drug_name):
    try:
        url = f'{find_drug_page(drug_name)}/overview/ic50'
        response = requests.get(url)
    except Exception as e:
        # print(f"Failed to retrieve the web page overview for drug {drug_name}, {e}")
        return {'name': "", 'target': "", 'pathway': ""}

    if response.ok:
        try:
            soup = BeautifulSoup(response.text, 'html.parser')
            compound = soup.find('h3', class_='emboldened')
            drug_target = compound.find_next_sibling('h4')
            drug_target_pathway = drug_target.find_next_sibling('h4')
            return {'name': compound, 'target': drug_target, 'pathway': drug_target_pathway}
        except Exception as e:
            # print(f"Failed to retrieve the web page for drug {drug_name}, {e}")
            return {'name': compound, 'target': "", 'pathway': ""}
    else:
        # print(f"Failed to retrieve the web page for drug {drug_name}. Status code: {response.status_code}")
        return {'name': "", 'target': "", 'pathway': ""}

In [9]:
def drug_info_ws_v2(drug_name): ### faster
    try:
        url = f'https://www.cancerrxgene.org/search?query={drug_name}'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        drug_info_table = soup.find_all('table')[0].find_all('td')
        drug_target = drug_info_table[2].text.strip()
        drug_target_pathway = drug_info_table[3].text.strip()
        return {'name': drug_name, 'target': drug_target, 'pathway': drug_target_pathway}
    except Exception as e:
        # print(f"Failed to retrieve the web page overview for drug {drug_name}, {e}")
        return {'name': "", 'target': "", 'pathway': ""}

In [10]:
### Drug features
not_avail_drug_lst = []
for n in range(N):
    drug_id = f"drug_{n+1}"
    for i in tqdm(range(len(interm_data['cell_id']))):
        name = interm_data[f'{drug_id}_name'][i]
        smiles = drug_smiles_data.loc[drug_smiles_data.drug_name == name, 'smiles'].values[0]
        interm_data[f'{drug_id}_SMILES'].append(smiles)
        drug_info_data = drug_info_ws_v2(name)
        if drug_info_data['target'] == "" and name not in not_avail_drug_lst:
            not_avail_drug_lst.append(name)
        interm_data[f'{drug_id}_target'].append(drug_info_data['target'])
        interm_data[f'{drug_id}_target_path'].append(drug_info_data['pathway'])

100%|████████████████████████████████████████████████████████████████████████████████████████| 692/692 [13:22<00:00,  1.16s/it]


In [11]:
interm_df = pd.DataFrame(interm_data)
print(interm_df.shape)
interm_df.head()

(692, 17)


,cell_id,mut_status,drug_1_name,drug_1_SMILES,drug_1_target,drug_1_target_path,drug_1_response,drug_2_name,drug_2_SMILES,drug_2_target,drug_2_target_path,drug_2_response,drug_3_name,drug_3_SMILES,drug_3_target,drug_3_target_path,drug_3_response
0,ACH-000001,"NOTCH1, NOTCH3, PIK3R1, PPP2R1A, TP53, TSC2, W...",KIN001-135,COC1=C(C=C2C(=C1)N=CN2C3=CC(=C(S3)C#N)OCC4=CC=...,IKK,"Other, kinases",0.072277,BRYOSTATIN-1,CCCC=CC=CC(=O)OC1C(=CC(=O)OC)CC2CC(OC(=O)CC(CC...,,,0.236236,GSK2606414,CN1C=C(C2=C(N=CN=C21)N)C3=CC4=C(C=C3)N(CC4)C(=...,PERK,Metabolism,0.236837
1,ACH-000002,"CDKN2A, CEBPA, KDM5C, NRAS, RPTOR",FORETINIB,COC1=CC2=C(C=CN=C2C=C1OCCCN3CCOCC3)OC4=C(C=C(C...,"MET, KDR, TIE2, VEGFR3/FLT4, RON, PDGFR, FGFR1...",RTK signaling,0.034317,KIN001-135,COC1=C(C=C2C(=C1)N=CN2C3=CC(=C(S3)C#N)OCC4=CC=...,IKK,"Other, kinases",0.046061,WYE-125132,CNC(=O)NC1=CC=C(C=C1)C2=NC3=C(C=NN3C4CCC5(CC4)...,mTOR,PI3K/MTOR signaling,0.158279
2,ACH-000004,"FGFR3, POLE, TP53, ZNF703",WYE-125132,CNC(=O)NC1=CC=C(C=C1)C2=NC3=C(C=NN3C4CCC5(CC4)...,mTOR,PI3K/MTOR signaling,0.057180,IMATINIB,CC1=C(C=C(C=C1)NC(=O)C2=CC=C(C=C2)CN3CCN(CC3)C...,"ABL, KIT, PDGFR","Other, kinases",0.069230,KOBE2602,C1=CC(=CC=C1NC(=S)NNC2=C(C=C(C=C2[N+](=O)[O-])...,RAS effector,RTK signaling,0.118252
3,ACH-000006,"ASXL1, FLT3, MAP3K1, NOTCH1, TP53, U2AF1",ABT-737,CN(C)CC[C@H](CSC1=CC=CC=C1)NC2=C(C=C(C=C2)S(=O...,,,0.047242,LINSITINIB,CC1(CC(C1)C2=NC(=C3N2C=CN=C3N)C4=CC5=C(C=C4)C=...,IGF1R,IGF1R signaling,0.056793,PIK-93,CC1=C(SC(=N1)NC(=O)C)C2=CC(=C(C=C2)Cl)S(=O)(=O...,PK3CG,PI3K/MTOR signaling,0.095844
4,ACH-000007,"ALK, BRAF, KRAS, MET, MUTYH, NOTCH3, RAD54L, R...",BORTEZOMIB,B([C@H](CC(C)C)NC(=O)[C@H](CC1=CC=CC=C1)NC(=O)...,Proteasome,Protein stability and degradation,0.306996,KOBE2602,C1=CC(=CC=C1NC(=S)NNC2=C(C=C(C=C2[N+](=O)[O-])...,RAS effector,RTK signaling,0.324368,WYE-125132,CNC(=O)NC1=CC=C(C=C1)C2=NC3=C(C=NN3C4CCC5(CC4)...,mTOR,PI3K/MTOR signaling,0.336791


In [12]:
len(not_avail_drug_lst)

14

In [17]:
### Some drug info is not available on the GDSC platform?
print(not_avail_drug_lst)

['ABT-737', 'BRYOSTATIN-1', 'QS-11', 'JNJ-38877605', 'ACETALAX', 'NU-7441', 'JQ1-(+)', 'VINCRISTINE', 'SINULARIN', 'T-0901317', 'SB-505124', 'SU-11274', 'EPOTHILONE-B', 'FILANESIB']


In [19]:
# interm_df.to_csv("/data/yingfei/cancer_data/llm_prompt_data/llm_interm_data_task2.csv")

In [8]:
### import interm_df to create the prompt_data
interm_df = pd.read_csv("/data/yingfei/cancer_data/llm_prompt_data/llm_interm_data_task2.csv", index_col = 0)
print(interm_df.shape)
interm_df.head()

(692, 17)


,cell_id,mut_status,drug_1_name,drug_1_SMILES,drug_1_target,drug_1_target_path,drug_1_response,drug_2_name,drug_2_SMILES,drug_2_target,drug_2_target_path,drug_2_response,drug_3_name,drug_3_SMILES,drug_3_target,drug_3_target_path,drug_3_response
0,ACH-000001,"NOTCH1, NOTCH3, PIK3R1, PPP2R1A, TP53, TSC2, W...",KIN001-135,COC1=C(C=C2C(=C1)N=CN2C3=CC(=C(S3)C#N)OCC4=CC=...,IKK,"Other, kinases",0.072277,BRYOSTATIN-1,CCCC=CC=CC(=O)OC1C(=CC(=O)OC)CC2CC(OC(=O)CC(CC...,NaN,NaN,0.236236,GSK2606414,CN1C=C(C2=C(N=CN=C21)N)C3=CC4=C(C=C3)N(CC4)C(=...,PERK,Metabolism,0.236837
1,ACH-000002,"CDKN2A, CEBPA, KDM5C, NRAS, RPTOR",FORETINIB,COC1=CC2=C(C=CN=C2C=C1OCCCN3CCOCC3)OC4=C(C=C(C...,"MET, KDR, TIE2, VEGFR3/FLT4, RON, PDGFR, FGFR1...",RTK signaling,0.034317,KIN001-135,COC1=C(C=C2C(=C1)N=CN2C3=CC(=C(S3)C#N)OCC4=CC=...,IKK,"Other, kinases",0.046061,WYE-125132,CNC(=O)NC1=CC=C(C=C1)C2=NC3=C(C=NN3C4CCC5(CC4)...,mTOR,PI3K/MTOR signaling,0.158279
2,ACH-000004,"FGFR3, POLE, TP53, ZNF703",WYE-125132,CNC(=O)NC1=CC=C(C=C1)C2=NC3=C(C=NN3C4CCC5(CC4)...,mTOR,PI3K/MTOR signaling,0.057180,IMATINIB,CC1=C(C=C(C=C1)NC(=O)C2=CC=C(C=C2)CN3CCN(CC3)C...,"ABL, KIT, PDGFR","Other, kinases",0.069230,KOBE2602,C1=CC(=CC=C1NC(=S)NNC2=C(C=C(C=C2[N+](=O)[O-])...,RAS effector,RTK signaling,0.118252
3,ACH-000006,"ASXL1, FLT3, MAP3K1, NOTCH1, TP53, U2AF1",ABT-737,CN(C)CC[C@H](CSC1=CC=CC=C1)NC2=C(C=C(C=C2)S(=O...,NaN,NaN,0.047242,LINSITINIB,CC1(CC(C1)C2=NC(=C3N2C=CN=C3N)C4=CC5=C(C=C4)C=...,IGF1R,IGF1R signaling,0.056793,PIK-93,CC1=C(SC(=N1)NC(=O)C)C2=CC(=C(C=C2)Cl)S(=O)(=O...,PK3CG,PI3K/MTOR signaling,0.095844
4,ACH-000007,"ALK, BRAF, KRAS, MET, MUTYH, NOTCH3, RAD54L, R...",BORTEZOMIB,B([C@H](CC(C)C)NC(=O)[C@H](CC1=CC=CC=C1)NC(=O)...,Proteasome,Protein stability and degradation,0.306996,KOBE2602,C1=CC(=CC=C1NC(=S)NNC2=C(C=C(C=C2[N+](=O)[O-])...,RAS effector,RTK signaling,0.324368,WYE-125132,CNC(=O)NC1=CC=C(C=C1)C2=NC3=C(C=NN3C4CCC5(CC4)...,mTOR,PI3K/MTOR signaling,0.336791


In [20]:
### Missing drug information about target and target pathway --> set as Unknown
# 'ABT-737'
# 'BRYOSTATIN-1'
# 'QS-11'
# 'JNJ-38877605'
# 'ACETALAX'
# 'NU-7441'
# 'JQ1-(+)'
# 'VINCRISTINE'
# 'SINULARIN'
# 'T-0901317'
# 'SB-505124'
# 'SU-11274'
# 'EPOTHILONE-B'
# 'FILANESIB'

In [89]:
### prompt data
prompt_data = {
    'cell_id': [],
    'prompt': [],
    'answer': []
}

In [90]:
for i in range(len(interm_df)):
    order = list(range(1, N+1)) 
    random.shuffle(order) ### shuffle the order of the drug options
    prompt = "Think step by step and decide the best drug option for the cell line with given mutations: [Drug Name], [Reasoning].\n"
    cell_drug_data = interm_df.iloc[i]
    mut_status = cell_drug_data['mut_status']
    cell_id = cell_drug_data['cell_id']
    idx = 0
    for n in order:
        idx += 1
        drug_name = cell_drug_data[f'drug_{n}_name']
        drug_SMILES = cell_drug_data[f'drug_{n}_SMILES']
        drug_SMILES = drug_SMILES if pd.notna(drug_SMILES) else "Unknown"
        drug_target = cell_drug_data[f'drug_{n}_target']
        drug_target = drug_target if pd.notna(drug_target) else "Unknown"
        drug_target_path = cell_drug_data[f'drug_{n}_target_path']
        drug_target_path = drug_target_path if pd.notna(drug_target_path) else "Unknown"
        prompt += f"Drug {idx}: The drug is {drug_name}. The drug SMILES structure is {drug_SMILES}. Drug target is {drug_target}. Drug target pathway is {drug_target_path}.\n"
    prompt += f"The mutations of the cell line are {mut_status}.\n"
    prompt += "Best drug option: ?"
    prompt_data['cell_id'].append(cell_id)
    prompt_data['prompt'].append(prompt)
    prompt_data['answer'].append(cell_drug_data[f'drug_1_name']) ### best drug option with the lowest AUDRC value

In [91]:
prompt_df = pd.DataFrame(prompt_data)
for i in range(3):
    print(prompt_df.prompt[i])
    print(prompt_df.answer[i])
    print()

Think step by step and decide the best drug option for the cell line with given mutations: [Drug Name], [Reasoning].
Drug 1: The drug is BRYOSTATIN-1. The drug SMILES structure is CCCC=CC=CC(=O)OC1C(=CC(=O)OC)CC2CC(OC(=O)CC(CC3CC(C(C(O3)(CC4CC(=CC(=O)OC)CC(O4)C=CC(C1(O2)O)(C)C)O)(C)C)OC(=O)C)O)C(C)O. Drug target is Unknown. Drug target pathway is Unknown.
Drug 2: The drug is KIN001-135. The drug SMILES structure is COC1=C(C=C2C(=C1)N=CN2C3=CC(=C(S3)C#N)OCC4=CC=CC=C4S(=O)(=O)C)OC. Drug target is IKK. Drug target pathway is Other, kinases.
Drug 3: The drug is GSK2606414. The drug SMILES structure is CN1C=C(C2=C(N=CN=C21)N)C3=CC4=C(C=C3)N(CC4)C(=O)CC5=CC(=CC=C5)C(F)(F)F. Drug target is PERK. Drug target pathway is Metabolism.
The mutations of the cell line are NOTCH1, NOTCH3, PIK3R1, PPP2R1A, TP53, TSC2, WHSC1L1.
Best drug option: ?
KIN001-135

Think step by step and decide the best drug option for the cell line with given mutations: [Drug Name], [Reasoning].
Drug 1: The drug is WYE-12513

In [92]:
### train_test_split
### train data
f = open("/home/yingfei/train_celllines_v1_1111.txt", "r")
train_cell_line = [line.strip() for line in f.readlines()]
print(len(train_cell_line))

### test data
f = open("/home/yingfei/test_celllines_v1_1111.txt", "r")
test_cell_line = [line.strip() for line in f.readlines()]
print(len(test_cell_line))

### train_prompt_data
train_prompt_data = prompt_df.loc[prompt_df.cell_id.isin(train_cell_line)].reset_index(drop = True)
print(train_prompt_data.shape)

### test_prompt_data
test_prompt_data = prompt_df.loc[prompt_df.cell_id.isin(test_cell_line)].reset_index(drop = True)
print(test_prompt_data.shape)

623
69
(623, 3)
(69, 3)


In [93]:
# train_prompt_data.to_csv("/data/yingfei/cancer_data/llm_prompt_data/train_prompt_data_task2.csv", index = False)
# test_prompt_data.to_csv("/data/yingfei/cancer_data/llm_prompt_data/test_prompt_data_task2.csv", index = False)